##### Import

In [26]:
import warnings
import papermill as pm
import scrapbook as sb
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from tqdm import tqdm
import shap
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LinearRegression, Ridge, Lasso
# from sklearn.pipeline import Pipeline
import os
import gc
import sys

# Filter out warning messages
warnings.filterwarnings('ignore')

# Set pandas display options
pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows', 10000)

# Set seaborn style
sns.set_style('whitegrid')

# Add the parent directory to sys.path
sys.path.insert(1, os.path.join(sys.path[0], '..'))

# Index and deciles for data slicing
idx = pd.IndexSlice



from pathlib import Path

# Paths to the downloaded datasets, model, and hyperparameters
data_dir = Path('data/')
model_dir = Path('model/')
best_hyperparams_dir = Path('best_hyperparams/')
study_dir = Path('study/')

# Create directories if they do not exist
data_dir.mkdir(parents=True, exist_ok=True)
model_dir.mkdir(parents=True, exist_ok=True)
best_hyperparams_dir.mkdir(parents=True, exist_ok=True)
study_dir.mkdir(parents=True, exist_ok=True)

In [27]:
# from pathlib import Path
# import pandas as pd
# from utils import rank_stocks_and_quantile
# # UNSEEN_KEY = '/data/YEAR_20220803_20230803'
# top = 250  # parameters -> papermill
# DATA_STORE = Path(f'data/{top}_dataset.h5')
# with pd.HDFStore(DATA_STORE) as store:
#     # unseen = store[UNSEEN_KEY]
#     print(store.keys())

In [28]:
import gc
import numpy as np
import pandas as pd
from pathlib import Path
from utils import rank_stocks_and_quantile
from sklearn.preprocessing import MinMaxScaler

# Parameters and data paths
top = 250
DATA_STORE = Path(f'data/{top}_dataset.h5')
dataset_keys = [
    '/data/YEAR_20200930_20220802',
    # '/data/YEAR_20181024_20200929',
    # '/data/YEAR_20161116_20181023',
    # '/data/YEAR_20141210_20161115'
]
target_string = 'TARGET_ret_fwd'
CHUNK_SIZE = 50000

def convert_dtype(chunk, feature_columns, dtype='float32'):
    """Converts the datatype of the specified columns."""
    chunk[feature_columns] = chunk[feature_columns].astype(dtype)
    return chunk

def handle_infinite_values(chunk, feature_columns):
    """Handle infinite values by replacing them with the maximum finite value."""
    max_val = np.finfo('float32').max
    chunk[feature_columns] = chunk[feature_columns].replace([np.inf, -np.inf], max_val)
    return chunk

def process_chunk(chunk, feature_columns, scaler=None):
    """Process a single chunk of data."""
    chunk = convert_dtype(chunk, feature_columns)
    chunk = handle_infinite_values(chunk, feature_columns)
    
    # Normalize with scaler if provided
    if scaler:
        chunk[feature_columns] = scaler.transform(chunk[feature_columns])
    
    return chunk

# Identify features and targets based on the first chunk
with pd.HDFStore(DATA_STORE) as store:
    first_chunk = store.select(dataset_keys[0], stop=CHUNK_SIZE)
    features = [col for col in first_chunk.columns if col.startswith('FEATURE_')]
    target = [col for col in first_chunk.columns if col.startswith('TARGET_')]

# Determine the scaler using the entire dataset for the identified features
scaler = MinMaxScaler()
for key in dataset_keys:
    with pd.HDFStore(DATA_STORE) as store:
        for chunk in store.select(key, chunksize=CHUNK_SIZE):
            # Convert dtype and handle infinite values
            chunk = convert_dtype(chunk, features)
            chunk = handle_infinite_values(chunk, features)
            scaler.partial_fit(chunk[features])

# Process and concatenate chunks
dataset = pd.DataFrame()
for key in dataset_keys:
    with pd.HDFStore(DATA_STORE) as store:
        for chunk in store.select(key, chunksize=CHUNK_SIZE):
            processed_chunk = process_chunk(chunk, features, scaler)
            dataset = pd.concat([dataset, processed_chunk], ignore_index=False)
            del processed_chunk
            gc.collect()

# Post-processing steps
dataset = rank_stocks_and_quantile(dataset, target_substring=target_string)
dataset.index.set_levels(dataset.index.levels[0].tz_localize(None), level=0, inplace=True)

In [29]:
dataset = dataset.head(10**4)

In [30]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from tqdm import tqdm
from joblib import Parallel, delayed

PADDING_VALUE = -1
MAX_LEN = None  # If you have a predefined value, set it here; otherwise, it gets calculated automatically.

def pad_sequence(inputs, padding_value=-1, max_len=None):
    if max_len is None:
        max_len = max([input.shape[0] for input in inputs])
    padded_inputs = []
    masks = []
    for input in inputs:
        pad_len = max_len - input.shape[0]
        padded_input = F.pad(input, (0, 0, 0, pad_len), value=padding_value)
        mask = torch.ones((input.shape[0], 1), dtype=torch.float)
        masks.append(
            torch.cat((mask, torch.zeros((pad_len, 1), dtype=torch.float)), dim=0)
        )
        padded_inputs.append(padded_input)
    return torch.stack(padded_inputs), torch.stack(masks)

def convert_to_torch(timestamp, data):
    feature_names = [col for col in data.columns if col.startswith('FEATURE_')]
    target_names = [col for col in data.columns if col.startswith('TARGET_')]
    
    inputs = torch.from_numpy(
                data[feature_names].values.astype(np.float32))
    labels = torch.from_numpy(
                data[target_names].values.astype(np.float32))

    padded_inputs, masks_inputs = pad_sequence(
            [inputs], padding_value=PADDING_VALUE, max_len=MAX_LEN)
    padded_labels, masks_labels = pad_sequence(
            [labels], padding_value=PADDING_VALUE, max_len=MAX_LEN)

    return {
        timestamp: (
            padded_inputs,
            padded_labels,
            masks_inputs
        )
    }

def get_era2data(df):
    # Group by the Timestamp index (level=0)
    res = Parallel(n_jobs=-1, prefer="threads")(
        delayed(convert_to_torch)(timestamp, data)
        for timestamp, data in tqdm(df.groupby(level=0)))
    
    era2data = {}
    for r in tqdm(res):
        era2data.update(r)
    return era2data

# # Assuming your DataFrame is named "dataset"
# timestamp2data_dataset = get_era2data(dataset)

In [31]:
import torch 

PADDING_VALUE = -1
MAX_LEN = 6000
FEATURE_DIM = len(features)
HIDDEN_DIM = 128
OUTPUT_DIM = len(target)
NUM_HEADS = 2
NUM_LAYERS = 2

device = "cuda" if torch.cuda.is_available() else "cpu"


from model import Transformer

def test_model():

    inputs = [
        torch.randint(0, 4, (5, FEATURE_DIM)).float(),
        torch.randint(0, 4, (3, FEATURE_DIM)).float(),
    ]
    labels = [
        torch.randint(0, 2, (5, OUTPUT_DIM)).float(),
        torch.randint(0, 2, (3, OUTPUT_DIM)).float(),
    ]

    padded_inputs, masks_inputs = pad_sequence(inputs, \
                                               padding_value=0, max_len=MAX_LEN)
    padded_labels, masks_labels = pad_sequence(labels, \
                                               padding_value=0, max_len=MAX_LEN)

    transformer = Transformer(
        input_dim=FEATURE_DIM,
        d_model=HIDDEN_DIM,
        output_dim=OUTPUT_DIM,
        num_heads=NUM_HEADS,
        num_layers=NUM_LAYERS,
        max_len=MAX_LEN,
    )

    with torch.no_grad():
        outputs = transformer(padded_inputs, masks_inputs)

    assert torch.isnan(outputs).sum() == 0
    assert outputs.shape[:2] == padded_inputs.shape[:2]
    assert outputs.shape[-1] == len(target)

    print("Input Shape:", padded_inputs.shape)
    print("Output Shape:", outputs.shape)

    del transformer
    del inputs, labels
    del padded_inputs, masks_inputs, padded_labels, masks_labels
    del outputs

    gc.collect()

test_model()

Input Shape: torch.Size([2, 6000, 586])
Output Shape: torch.Size([2, 6000, 12])


In [32]:
# pearsonr in torch differentiable
def pearsonr(x, y):
    mx = x.mean()
    my = y.mean()
    xm, ym = x - mx, y - my
    r_num = torch.sum(xm * ym)
    r_den = torch.sqrt(torch.sum(xm ** 2) * torch.sum(ym ** 2))
    r = r_num / r_den
    return r

In [33]:
def calculate_loss(outputs, criterion, padded_labels, masks_inputs, \
                padded_inputs=None, target_weight_softmax=None):

    # MSE on all targets; additionally, on primary target
    if target_weight_softmax is not None:
        _mse = criterion(
            outputs * masks_inputs * target_weight_softmax,
            padded_labels * masks_inputs * target_weight_softmax
        ) * 0.1

    else:
        _mse = criterion(outputs * masks_inputs, padded_labels * masks_inputs) * 0.1

    _mse += criterion(outputs[:, 0] * masks_inputs, padded_labels[:, 0] * masks_inputs)

    # Corr with only primary target; adjust as needed
    corr = pearsonr(
        outputs[0][:, 0][masks_inputs.view(-1).nonzero()].view(-1, 1),
        padded_labels[0][:, 0][masks_inputs.view(-1).nonzero()].view(-1, 1),
    )

    loss = _mse - corr #+ some_complex_constraints
    return loss, _mse, corr

# Training loop
def train_on_batch(transformer, criterion, optimizer, batch):

    padded_inputs = batch[0].to(device=device)
    padded_labels = batch[1].to(device=device)
    masks_inputs = batch[2].to(device=device)

    optimizer.zero_grad()

    outputs = transformer(padded_inputs / 4.0, masks_inputs)
    # print(outputs)

    target_weight_softmax = None
    #random_weights = torch.rand(padded_labels.shape[-1], device=device)
    #target_weight_softmax = F.softmax(random_weights)

    loss, _mse, _corr = calculate_loss(outputs, criterion, padded_labels, masks_inputs, \
                                       target_weight_softmax=target_weight_softmax)
    loss.backward()
    optimizer.step()
    return loss.item(), _mse.item(), _corr.item()


def evaluate_on_batch(transformer, criterion, batch):

    padded_inputs = batch[0].to(device=device)
    padded_labels = batch[1].to(device=device)
    masks_inputs = batch[2].to(device=device)

    transformer.eval()
    with torch.no_grad():
        outputs = transformer(padded_inputs / 4.0, masks_inputs)
        # print(outputs)
        loss, _mse, _corr = calculate_loss(outputs, criterion, padded_labels, masks_inputs)
        
        # Convert outputs to numpy
        preds = outputs[0][masks_inputs.view(-1).nonzero()].squeeze(1).cpu().numpy()
        # print(preds)

    return loss.item(), _mse.item(), _corr.item(), preds


def metrics_on_batch(era_scores):
    era_scores = pd.Series(era_scores)
    
    # Calculate metrics
    mean_correlation = np.mean(era_scores)
    std_deviation = np.std(era_scores)
    sharpe_ratio = mean_correlation / std_deviation
    max_dd = (era_scores.cummax() - era_scores).max() # from calculate_metrics

    # Smart Sharpe: Modified Sharpe ratio that also considers the instability of scores over time,
    # penalizing models with high score instability even if their mean score is high
    smart_sharpe = mean_correlation / (std_deviation + np.std(era_scores.diff()))
    
    # Autocorrelation: Measure of the correlation of the series with a lagged version of itself
    autocorrelation = era_scores.autocorr()

    metrics = pd.Series({
        'mean_correlation': mean_correlation,
        'std_deviation': std_deviation,
        'sharpe_ratio': sharpe_ratio,
        'smart_sharpe': smart_sharpe,
        'autocorrelation': autocorrelation,
        'max_dd': max_dd, # added from calculate_metrics
        'min_correlation': era_scores.min(), # added from calculate_metrics
        'max_correlation': era_scores.max(), # added from calculate_metrics
    })

    # Cleanup
    _ = gc.collect()
    
    return metrics

In [34]:
from tqdm.auto import tqdm

def train_model(transformer, criterion, optimizer, scheduler, \
                num_epochs, patience, train_loader, val_loader, is_lr_scheduler=True):
    best_loss = float('inf')
    best_corr = None
    best_model = None
    best_outputs = None
    no_improve_epoch = 0

    epoch_progress = tqdm(range(num_epochs), desc="Epochs", position=0, leave=False)

    for epoch in epoch_progress:
        total_loss = []
        total_corr = []

        # Training
        for era_num in tqdm(train_loader, desc="Training", leave=False, position=1):
            batch = train_loader[era_num]
            loss, _mse, _corr = train_on_batch(transformer, criterion, optimizer, batch)
            total_loss.append(loss)
            total_corr.append(_corr)

        # Adjust learning rate if is_lr_scheduler is True
        if is_lr_scheduler:
            scheduler.step()

        # Validation
        transformer.eval()
        val_total_loss = []
        val_total_corr = []
        val_total_outputs = {}
        with torch.no_grad():
            for era_num in tqdm(val_loader, desc="Validation", leave=False, position=2):
                batch = val_loader[era_num]
                loss, _mse, _corr, outputs = evaluate_on_batch(transformer, criterion, batch)
                # print(outputs)
                val_total_loss.append(loss)
                val_total_corr.append(_corr)
                val_total_outputs[era_num] = outputs

        # Early stopping check
        val_loss = np.mean(val_total_loss)
        if val_loss < best_loss:
            best_loss = val_loss
            best_corr = val_total_corr.copy()
            best_model = transformer.state_dict().copy()
            best_outputs = val_total_outputs.copy()
            no_improve_epoch = 0
        else:
            no_improve_epoch += 1
            if no_improve_epoch >= patience:
                epoch_progress.set_description(f'Early stopping at epoch {epoch+1}')
                epoch_progress.refresh()
                break

        torch.cuda.empty_cache()
        _ = gc.collect()

    # Save the best model state
    torch.save(best_model, data_dir / "transformer_best.pth")

    return transformer, best_corr, best_outputs # best_outputs for future use


In [35]:
import optuna
from torch.optim.lr_scheduler import StepLR
from utils import CustomBackwardMultipleTimeSeriesCV
import torch.nn as nn

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import math


NUM_EPOCHS = 1
PATIENCE = 5

def objective(trial, dataset=dataset):
    print(f"\n--- Starting Trial: {trial.number + 1} ---")
    
    # Instantiate CV object inside the objective function
    cv = CustomBackwardMultipleTimeSeriesCV(dataset, train_period_length=10, 
                                            test_period_length=21, 
                                            lookahead=1,  # Starting value
                                            date_idx='date')
    
    # Suggest hyperparameters
    num_heads = trial.suggest_int("num_heads", 1, 5)
    hidden_dim = trial.suggest_int("hidden_dim", 64, 256, step=2)
    num_layers = trial.suggest_int("num_layers", 1, 5)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)

    transformer = Transformer(
        input_dim=FEATURE_DIM,
        d_model=hidden_dim,
        output_dim=OUTPUT_DIM,
        num_heads=num_heads,
        num_layers=num_layers,
    )
    transformer.to(device=device)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(transformer.parameters(), lr=lr)
    scheduler = StepLR(optimizer, step_size=100, gamma=0.1)

    sharpe_ratios = []

    # Update the CV's lookahead based on the trial's suggestion
    look_ahead = 1
    cv.update_lookahead(look_ahead)

    # Loop through train and test splits from the CV
    for train_idx, test_idx in cv:
        train_data = dataset.iloc[train_idx]
        test_data = dataset.iloc[test_idx]

        # Convert train_data and test_data to the desired format
        era2data_train = get_era2data(train_data)
        era2data_validation = get_era2data(test_data)

        transformer, best_corr, outputs = train_model(transformer, criterion, optimizer, scheduler,
                                                      NUM_EPOCHS, PATIENCE, era2data_train, 
                                                      era2data_validation, is_lr_scheduler=True)

        metrics = metrics_on_batch(best_corr)
        sharpe_ratios.append(metrics['sharpe_ratio'])

    trial.set_user_attr("transformer", transformer)
    
    return -np.mean(sharpe_ratios)  # average negative sharpe ratio over all splits

# Callback to save the best model and its hyperparameters after each trial
def callback(study, trial):
    print(f"\n--- Trial {trial.number + 1} finished ---")
    print(f"Value: {trial.value} and parameters: {trial.params}")
    print(f"Best is trial {study.best_trial.number} with value: {study.best_trial.value}\n")
    
    if study.best_trial.number == trial.number:
        best_model = trial.user_attrs["transformer"]
        torch.save(best_model, model_dir / "best_model.pth")
        with open(best_hyperparams_dir / "best_params.json", 'w') as f:
            json.dump(trial.params, f)

study = optuna.create_study(study_name='Maximizing the Sharpe', direction='minimize',
                            storage=f'sqlite:///{study_dir}/study.db', load_if_exists=True)
study.optimize(objective, n_trials=25, callbacks=[callback])

gc.collect()

[I 2023-10-06 17:23:48,281] A new study created in RDB with name: Maximizing the Sharpe



--- Starting Trial: 1 ---


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

Epochs:   0%|          | 0/1 [00:00<?, ?it/s]

Training:   0%|          | 0/10 [00:00<?, ?it/s]

[W 2023-10-06 17:23:49,184] Trial 0 failed with parameters: {'num_heads': 4, 'hidden_dim': 240, 'num_layers': 5, 'learning_rate': 0.000201614661048267} because of the following error: RuntimeError('The size of tensor a (12) must match the size of tensor b (36) at non-singleton dimension 2').
Traceback (most recent call last):
  File "/home/sayem/anaconda3/lib/python3.11/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_143574/1737206000.py", line 60, in objective
    transformer, best_corr, outputs = train_model(transformer, criterion, optimizer, scheduler,
                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_143574/2781623084.py", line 20, in train_model
    loss, _mse, _corr = train_on_batch(transformer, criterion, optimizer, batch)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 

RuntimeError: The size of tensor a (12) must match the size of tensor b (36) at non-singleton dimension 2

In [ ]:
STOP

In [ ]:
from utils import CustomBackwardMultipleTimeSeriesCV

look_ahead = 1

cv = CustomBackwardMultipleTimeSeriesCV(dataset, train_period_length = 10, 
                                    test_period_length = 21, 
                                    lookahead=1,  # Starting value; we'll adjust it next.
                                    date_idx='date')

# look_ahead = 1

# Update the CV's lookahead based on the trial's suggestion
cv.update_lookahead(look_ahead)

In [ ]:
for train_idx, val_idx in cv:
    # Dynamic target based on the suggested lookahead
    label = f'TARGET_ret_fwd_{look_ahead:02d}d_rank_quantiled'

    train = dataset.loc[train_idx]
    validation = dataset.loc[val_idx]

    era2data_train = get_era2data(train)
    era2data_validation = get_era2data(validation)
    # train_labels = dataset.loc[train_idx, label]

    # val_features = dataset.loc[val_idx, features]
    # val_labels = dataset.loc[val_idx, label]

    break